# IDS Challenge
# Sub-Project 03: Robot Inverse Kinematics
## Results 

### Group: ___
### Tutor: ___

This is your workspace with some provided functions. Read the task instruction and implement all solutions here. See `README_EN.ipynb` first.

# Utils and Scene Definitions

Execute the cells below to enable the definition of the scene and make the util functions available.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import math

# --- Global Parameters for the Robot and Environment ---
L1 = 6.0  # Length of the first robot arm link
L2 = 6.0  # Length of the second robot arm link
L3 = 1.0  # Length of the rope/link holding the gripper attachment point

# Robot Gripper Parameters
GRIPPER_RADIUS = 0.5 # Radius of the robot's gripper (blue sphere/circle for collision)

# Manipulable Object Parameters (the one to be picked and placed)
MANIPULABLE_OBJECT_WIDTH = 2 * GRIPPER_RADIUS
MANIPULABLE_OBJECT_HEIGHT = 2 * GRIPPER_RADIUS

# --- MODIFIED: Obstacles reconfigured to create a horizontal gap ---
# This setup forces the robot to move through the gap between the two obstacles.
OBSTACLES = [
    # Lower barrier (floor of the gap)
    {"x_center": 4.5, "width": 1.0, "height": 3.5, "y_min": 0.0},
    {"x_center": 7.0, "width": 1.0, "height": 3.0, "y_min": 0.0},
    # Upper barrier (ceiling of the gap)
    {"x_center": 5.0, "width": 2.0, "height": 2.0, "y_min": 6.5}
]

# --- Pick and Place Station Parameters ---
PICK_STATION = {"x_center": 2.0, "width": 2.0, "height": 1.0, "y_min": 0.0}
PLACE_STATION = {"x_center": 9.5, "width": 2.0, "height": 1.0, "y_min": 0.0}
PICK_TARGET = np.array([2.0, 1.0])
PLACE_TARGET = np.array([9.5, 1.0]) 

# Initial position of the manipulable object (bottom-center) on the pick station
INITIAL_MANIPULABLE_OBJECT_POS = np.array([PICK_STATION["x_center"], PICK_STATION["y_min"] + PICK_STATION["height"]])

# --- Global State Variables for Animation ---
# These are used to store the pre-calculated path and track the state during the animation.
alpha_path = []
beta_path = []
x_path_dense = []
y_path_dense = []
object_is_picked = False
animation_running = True
# We will draw the object's position based on this variable, which is updated each frame.
current_manipulable_object_pos = INITIAL_MANIPULABLE_OBJECT_POS.copy()

In [ ]:
def robot_direct_kinematics(alpha, beta):
    """
    Calculates forward kinematics of the two-link robot arm using the
    same angle convention as inverse kinematics:
    alpha = shoulder angle q1
    beta = elbow angle q2

    Returns points P, Q, G, GC, GB as numpy arrays.
    """

    # Position of joint P (end of first link)
    P = L1 * np.array([np.cos(alpha), np.sin(alpha)])

    # Position of joint Q (end of second link / arm end)
    Q = P + L2 * np.array([np.cos(alpha + beta), np.sin(alpha + beta)])

    # Position of point G (bottom of rope)
    G = Q + L3 * np.array([0, -1])

    # Position of point GC (bottom of object, first radius offset)
    GC = G + GRIPPER_RADIUS * np.array([0, -1])

    # Position of point GB (bottom of object, second radius offset)
    GB = GC + GRIPPER_RADIUS * np.array([0, -1])

    return P, Q, G, GC, GB


# Visualization

Execute the cells below to make the visualization and animation available.

In [ ]:
def robot_inverse_kinematics(target_GB):
    pass

In [ ]:
def robot_path_points(target_points):
    """
    Converts a list of target [x, y] points into separate NumPy arrays for x and y coordinates,
    interpolating points between targets to create a smooth path.
    """
    if not target_points:
        return np.array([]), np.array([])

    x_coords = [target_points[0][0]]
    y_coords = [target_points[0][1]]

    for i in range(len(target_points) - 1):
        d = np.sqrt((target_points[i][0] - target_points[i+1][0])**2 + \
                    (target_points[i][1] - target_points[i+1][1])**2)
        num_points = max(2, math.ceil(d / (GRIPPER_RADIUS / 2)))
        x_coords.extend(np.linspace(target_points[i][0], target_points[i+1][0], num_points + 1)[1:])
        y_coords.extend(np.linspace(target_points[i][1], target_points[i+1][1], num_points + 1)[1:])

    return np.array(x_coords), np.array(y_coords)

def check_collision_circle_rect(circle_center, circle_radius, rect_x_center, rect_width, rect_height, rect_y_min_val):
    """
    Checks for collision between a circle and a rectangle.
    """
    rect_x_min = rect_x_center - rect_width / 2
    rect_x_max = rect_x_center + rect_width / 2
    rect_y_min = rect_y_min_val
    rect_y_max = rect_y_min_val + rect_height

    closest_x = np.clip(circle_center[0], rect_x_min, rect_x_max)
    closest_y = np.clip(circle_center[1], rect_y_min, rect_y_max)

    distance_sq = (circle_center[0] - closest_x)**2 + \
                  (circle_center[1] - closest_y)**2

    return distance_sq < circle_radius**2

def intersect_segments(p1, p2, p3, p4):
    """
    Checks if two line segments (p1,p2) and (p3,p4) intersect.
    """
    def orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - \
              (q[0] - p[0]) * (r[1] - q[1])
        if val == 0: return 0
        return 1 if val > 0 else 2

    def on_segment(p, q, r):
        return (q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and
                q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1]))

    o1 = orientation(p1, p2, p3)
    o2 = orientation(p1, p2, p4)
    o3 = orientation(p3, p4, p1)
    o4 = orientation(p3, p4, p2)

    if o1 != 0 and o2 != 0 and o3 != 0 and o4 != 0 and o1 != o2 and o3 != o4:
        return True
    if o1 == 0 and on_segment(p1, p3, p2): return True
    if o2 == 0 and on_segment(p1, p4, p2): return True
    if o3 == 0 and on_segment(p3, p1, p4): return True
    if o4 == 0 and on_segment(p3, p2, p4): return True

    return False

def check_line_rect_collision(p1, p2, rect_x_center, rect_width, rect_height, rect_y_min_val):
    """
    Checks for collision between a line segment (p1, p2) and a rectangle.
    """
    rect_x_min = rect_x_center - rect_width / 2
    rect_x_max = rect_x_center + rect_width / 2
    rect_y_min = rect_y_min_val
    rect_y_max = rect_y_min_val + rect_height

    if (rect_x_min <= p1[0] <= rect_x_max and rect_y_min <= p1[1] <= rect_y_max) or \
       (rect_x_min <= p2[0] <= rect_x_max and rect_y_min <= p2[1] <= rect_y_max):
        return True

    rect_corners = [
        np.array([rect_x_min, rect_y_min]),
        np.array([rect_x_max, rect_y_min]),
        np.array([rect_x_max, rect_y_max]),
        np.array([rect_x_min, rect_y_max])
    ]

    if intersect_segments(p1, p2, rect_corners[0], rect_corners[1]): return True
    if intersect_segments(p1, p2, rect_corners[1], rect_corners[2]): return True
    if intersect_segments(p1, p2, rect_corners[2], rect_corners[3]): return True
    if intersect_segments(p1, p2, rect_corners[3], rect_corners[0]): return True

    return False

def robot_draw(ax, alpha, beta, current_manipulable_object_pos_for_drawing):
    """
    Draws the robot, obstacles, pick/place stations, and the manipulable object onto the provided axes.
    """
    # Clear previous drawings from the axis
    ax.clear()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_aspect('equal')

    all_x_coords = [obs["x_center"] + obs["width"]/2 for obs in OBSTACLES] + \
                   [PICK_STATION["x_center"] + PICK_STATION["width"]/2,
                    PLACE_STATION["x_center"] + PLACE_STATION["width"]/2]
    max_x_val = max(all_x_coords)

    all_y_coords = [obs["y_min"] + obs["height"] for obs in OBSTACLES] + \
                   [PICK_STATION["y_min"] + PICK_STATION["height"],
                    PLACE_STATION["y_min"] + PLACE_STATION["height"]]
    max_y_val = max(all_y_coords)

    ax.set_xlim(-L1, max(L1 + L2 + 2, max_x_val + 2.0))
    ax.set_ylim(-1.0, max(L1 + L2 + 2, max_y_val + 2.0))

    baseline_x = np.array([-0.5, max(max_x_val + 2.0, 12.0)])
    baseline_y = np.array([0.0, 0.0])
    ax.plot(baseline_x, baseline_y, '-k')

    NORMAL_LINK_COLOR = [0.2, 0.2, 0.8]
    COLLISION_COLOR = [1.0, 0.0, 0.0]
    STATION_COLOR = [0.6, 0.6, 0.6]
    MANIPULABLE_OBJECT_COLOR = [1.0, 0.0, 0.0]

    link1_color = NORMAL_LINK_COLOR
    link2_color = NORMAL_LINK_COLOR
    link3_color = NORMAL_LINK_COLOR
    robot_gripper_color = [0.0, 0.0, 1.0]

    collided_obstacle_indices = set()
    overall_collision_detected = False

    O = np.array([0.0, 0.0])
    P, Q, G, GC, GB_fk = robot_direct_kinematics(alpha, beta)

    for idx, obs in enumerate(OBSTACLES):
        if check_collision_circle_rect(GC, GRIPPER_RADIUS, obs["x_center"], obs["width"], obs["height"], obs["y_min"]):
            robot_gripper_color = COLLISION_COLOR
            collided_obstacle_indices.add(idx)
            overall_collision_detected = True
        if check_line_rect_collision(O, P, obs["x_center"], obs["width"], obs["height"], obs["y_min"]):
            link1_color = COLLISION_COLOR
            collided_obstacle_indices.add(idx)
            overall_collision_detected = True
        if check_line_rect_collision(P, Q, obs["x_center"], obs["width"], obs["height"], obs["y_min"]):
            link2_color = COLLISION_COLOR
            collided_obstacle_indices.add(idx)
            overall_collision_detected = True
        if check_line_rect_collision(Q, G, obs["x_center"], obs["width"], obs["height"], obs["y_min"]):
            link3_color = COLLISION_COLOR
            collided_obstacle_indices.add(idx)
            overall_collision_detected = True

    for idx, obs in enumerate(OBSTACLES):
        obstacle_x = obs["x_center"] + obs["width"] / 2 * np.array([-1.0, 1.0, 1.0, -1.0, -1.0])
        obstacle_y = np.array([obs["y_min"], obs["y_min"], obs["y_min"] + obs["height"], obs["y_min"] + obs["height"], obs["y_min"]])
        obstacle_fill_color = COLLISION_COLOR if idx in collided_obstacle_indices else [0.0, 1.0, 0.0]
        ax.fill_between(obstacle_x, obstacle_y, facecolor=obstacle_fill_color, alpha=0.7)
        ax.plot(obstacle_x, obstacle_y, '-k')

    pick_x = PICK_STATION["x_center"] + PICK_STATION["width"] / 2 * np.array([-1.0, 1.0, 1.0, -1.0, -1.0])
    pick_y = np.array([PICK_STATION["y_min"], PICK_STATION["y_min"], PICK_STATION["y_min"] + PICK_STATION["height"], PICK_STATION["y_min"] + PICK_STATION["height"], PICK_STATION["y_min"]])
    ax.fill_between(pick_x, pick_y, facecolor=STATION_COLOR, alpha=0.8)
    ax.plot(pick_x, pick_y, '-k')
    ax.text(PICK_STATION["x_center"], PICK_STATION["y_min"] + PICK_STATION["height"] / 2, "PICK",
            horizontalalignment='center', verticalalignment='center', color='black', fontsize=10)

    place_x = PLACE_STATION["x_center"] + PLACE_STATION["width"] / 2 * np.array([-1.0, 1.0, 1.0, -1.0, -1.0])
    place_y = np.array([PLACE_STATION["y_min"], PLACE_STATION["y_min"], PLACE_STATION["y_min"] + PLACE_STATION["height"], PLACE_STATION["y_min"] + PLACE_STATION["height"], PLACE_STATION["y_min"]])
    ax.fill_between(place_x, place_y, facecolor=STATION_COLOR, alpha=0.8)
    ax.plot(place_x, place_y, '-k')
    ax.text(PLACE_STATION["x_center"], PLACE_STATION["y_min"] + PLACE_STATION["height"] / 2, "PLACE",
            horizontalalignment='center', verticalalignment='center', color='black', fontsize=10)

    rect_x_coords = current_manipulable_object_pos_for_drawing[0] + MANIPULABLE_OBJECT_WIDTH / 2 * np.array([-1.0, 1.0, 1.0, -1.0, -1.0])
    rect_y_coords = np.array([current_manipulable_object_pos_for_drawing[1], current_manipulable_object_pos_for_drawing[1],
                              current_manipulable_object_pos_for_drawing[1] + MANIPULABLE_OBJECT_HEIGHT,
                              current_manipulable_object_pos_for_drawing[1] + MANIPULABLE_OBJECT_HEIGHT,
                              current_manipulable_object_pos_for_drawing[1]])
    ax.fill_between(rect_x_coords, rect_y_coords, facecolor=MANIPULABLE_OBJECT_COLOR, alpha=0.9)
    ax.plot(rect_x_coords, rect_y_coords, '-k')

    ax.plot(np.array([O[0], P[0]]), np.array([O[1], P[1]]), color=link1_color)
    ax.plot(np.array([P[0], Q[0]]), np.array([P[1], Q[1]]), color=link2_color)
    ax.plot(np.array([Q[0], G[0]]), np.array([Q[1], G[1]]), color=link3_color)

    t = np.linspace(0.0, 2 * np.pi, 100)
    circle_x = GC[0] + GRIPPER_RADIUS * np.cos(t)
    circle_y = GC[1] + GRIPPER_RADIUS * np.sin(t)
    ax.fill_between(circle_x, circle_y, facecolor=robot_gripper_color)
    ax.plot(circle_x, circle_y, '-k')

    return overall_collision_detected, GB_fk


In [ ]:
def animate(frame_num):
    global object_is_picked, current_manipulable_object_pos, animation_running, ani

    if not animation_running:
        ani.event_source.stop()
        return

    # Joint angles and gripper position
    current_alpha = alpha_path[frame_num]
    current_beta = beta_path[frame_num]
    _, _, _, _, GB_fk = robot_direct_kinematics(current_alpha, current_beta)

    # Distance-based pick/place logic
    if not object_is_picked and np.linalg.norm(GB_fk - PICK_TARGET) < 0.1:
        object_is_picked = True
        print(f"[Frame {frame_num}] PICKED object at {GB_fk}")
    elif object_is_picked and np.linalg.norm(GB_fk - PLACE_TARGET) < 0.1:
        object_is_picked = False
        current_manipulable_object_pos = PLACE_TARGET.copy()
        print(f"[Frame {frame_num}] PLACED object at {PLACE_TARGET}")

    if object_is_picked:
        current_manipulable_object_pos = GB_fk.copy()

    collision, _ = robot_draw(ax, current_alpha, current_beta, current_manipulable_object_pos)

    title = f"Frame {frame_num+1}/{len(alpha_path)} | Object Picked: {object_is_picked}"
    if collision:
        title = "COLLISION DETECTED! Animation Halted."
        animation_running = False
    ax.set_title(title)

def run_pick_and_place_animation(waypoints):
    """
    Takes a list of waypoints and runs the full pick-and-place animation.
    """
    global fig, ax, ani, alpha_path, beta_path, x_path_dense, y_path_dense
    global object_is_picked, animation_running, current_manipulable_object_pos
    global PICK_TARGET_FROM_WAYPOINTS, PLACE_TARGET_FROM_WAYPOINTS

    # --- 1. State Initialization and Validation ---
    alpha_path, beta_path, x_path_dense, y_path_dense = [], [], [], []
    object_is_picked = False
    animation_running = True
    current_manipulable_object_pos = INITIAL_MANIPULABLE_OBJECT_POS.copy()

    if len(waypoints) < 4:
        print("Error: At least 4 waypoints are needed for a pick and place task (e.g., Hover-Pick, Pick, Hover-Place, Place).")
        return

    # --- 2. Path Pre-calculation ---
    print("Interpolating path from waypoints...")
    x_path_dense, y_path_dense = robot_path_points(waypoints)

    print("Pre-calculating Inverse Kinematics for the entire path...")
    for i in range(len(x_path_dense)):
        target_gb = np.array([x_path_dense[i], y_path_dense[i]])
        alpha, beta = robot_inverse_kinematics(target_gb)
        if alpha is None:
            print(f"ERROR: Path is unreachable. IK failed at point {i+1} ({target_gb[0]:.2f}, {target_gb[1]:.2f}).")
            return
        alpha_path.append(alpha)
        beta_path.append(beta)
            
    print("IK pre-calculation successful. Path is reachable.")

    # --- 3. Animation Setup and Execution ---
    print("Starting animation...")
    fig, ax = plt.subplots(figsize=(10, 8))
    ani = FuncAnimation(fig, animate, frames=len(x_path_dense), repeat=False, interval=50)
    
    display(HTML(ani.to_jshtml()))
    plt.close(fig)


# Task 2.1 Inverse Kinematics

## Determining Joint Angles from End-Effector Position

To move the robots end-effector point $T$ to a desired target position ($x_T$, $y_T$) we need to calculate the **joint angles** ($\alpha$, $\beta$). This process is known as **inverse kinematics**.

The fundamental task is to reverse the relationships established in direct kinematics. Based on the provided direct kinematics, points $R$, $M$, and $T$ are derived from point $Q$ by simple vertical offsets involving the rope length $L_3$ and the object's radius $r$. Therefore, the first step in inverse kinematics is to determine the coordinates of point $Q = (Q_x, Q_y)$ from the given target point $T$.

### 1. Determining the Effective Target Point $Q$

Given the coordinates of the target point $T = (x_T, y_T)$:

The vertical offset from $Q$ to $T$ is $L_3 + 2r$. To find $Q$'s coordinates, we simply reverse this offset:

$$Q_x = x_T$$
$$Q_y = y_T + (L_3 + 2r)$$

Now, $Q = (Q_x, Q_y)$ serves as our effective target point for the two-link arm ($L_1$, $L_2$).

### 2. Calculating the Elbow Angle ($\beta$)

From direct kinematics the formula for the end-effector position is already known:

$$Q_x = L_1 \cos(\alpha) + L_2 \cos(\alpha + \beta)$$
$$Q_y = L_1 \sin(\alpha) + L_2 \sin(\alpha + \beta)$$

Aiming to eliminate $\alpha$ from the equation we square and add both equations which in the end leads to: 

$$Q_x^2 + Q_y^2 = L_1^2 + L_2^2 + 2 L_1 L_2 \cos(\beta)$$

Rearranging to solve for $\cos(\beta)$:

$$\cos(\beta) = \frac{Q_x^2 + Q_y^2 - L_1^2 - L_2^2}{2 L_1 L_2}$$

To find $\beta$, we use the arctangent function. Note that for a given $\cos(\beta)$, there are generally two possible solutions for $\beta$ (corresponding to "elbow up" and "elbow down"). To achieve the "elbow down" configuration (as often desired for stability or workspace constraints), we select the negative value for $\sin(\beta)$:

$$\sin(\beta) = -\sqrt{1 - \cos^2(\beta)}$$

Then, $\beta$ is calculated using the `atan2` function, which correctly places the angle in the appropriate quadrant:

$$\beta = \text{atan2}(\sin(\beta), \cos(\beta))$$

**Important Note on Reachability**: If the calculated value of $\cos(\beta)$ is outside the range $[-1, 1]$, the target point $Q$ is beyond the robot's physical reach.

### 3. Calculating the Shoulder Angle ($\alpha$)

With $\beta$ determined, we can now compute the shoulder joint angle $\alpha$ using a geometric approach based on triangle construction. First, compute the angle of the vector from the origin to point $Q = (Q_x, Q_y)$:
$$\text{atan2}(Q_y,Q_x)$$

Next, compute a correction angle based on the triangle formed by the two links and the vector to $Q$. This correction accounts for how the second link (of length $L_2$) bends at the elbow with angle $\gamma$:
$$\text{atan2}(L_2\sin⁡(\beta),L_1+L_2\cos⁡(\beta))$$

The shoulder angle $\alpha$ is then given by subtracting the correction from the overall direction to $Q$:
$$\alpha = \text{atan2}(Q_y, Q_x) - \text{atan2}(L_2 \sin(\beta), L_1 + L_2 \cos(\beta))$$

In [ ]:
def robot_inverse_kinematics(target_GB):
    
    """
    put your code here
    """

    return alpha, beta


Now you can check if your inverse kinematics is implemented correct by inserting a list of two values where the first is the endeffectors x position, the second is the y position.

In [ ]:
point = ...

alpha, beta = robot_inverse_kinematics(point)
fig, ax = plt.subplots(figsize=(8, 6))
_, _ = robot_draw(ax, alpha, beta, INITIAL_MANIPULABLE_OBJECT_POS)

## Task 2.2 Guiding the Robot

In this section, you will define the path for the robot's gripper using a list of **waypoints**. Each waypoint is a specific $(x, y)$ coordinate that the gripper needs to reach. The robot system automatically interpolates linearly between the waypoints you specify.

The key idea is simple:

1.  **Define Waypoints to the Object:**
    You need to create a sequence of points that guide the robot's gripper **above** the object you want to pick, and then **down to** the object itself.

2.  **Automatic Picking:**
    Once the gripper successfully reaches the `pick_location`, the robot system will automatically detect the object and simulate picking it up. You don't need to write code for the grasping action itself.

3.  **Define Waypoints to the Drop-off Spot:**
    After picking, you'll define another sequence of points to move the robot, with the object, **above** the `place_location` (where you want to drop it), and then **down to** that spot. Make sure to plan your path to avoid obstacles.

4.  **Automatic Placing:**
    When the gripper reaches the `place_location`, the robot system will automatically simulate releasing the object, and it will stay there.

Your task is to define the `waypoints` list with these `[x, y]` coordinates. Think of it as drawing the path the gripper should follow. Then run the function `run_pick_and_place_animation(my_waypoints)` to see the animation.

---

In [ ]:
"""
put your code here
"""